# NEW Version


# Packages Unloading


In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
from Pacakging_prime import activity_time_check, hourly_activity_time_check, meat_pieces_check, get_best_plan, hourly_total_position_stdev, one_meat_checking


In [1]:
import pandas as pd
import math
names = locals()
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import statistics
# from PackagingToll import *
import pyodbc as odbc

# Reading the Files


#### Assumptions:

- Excluding byproducts

- Excluded offa

- Excluded the following activities:

  -- 6718, Box Room

  -- 3-digit activity codes

- Assuming the duration is for boxes not by pieces

- Products need to be completed by priority


In [86]:
# Sequence table
# seq=pd.read_excel("Files/Activities.xlsx", sheet_name='Product Seqeunce')
server = 'cmpcsb01'
database = 'conestoga'
username = 'remotequery'
password = "excel"
cnxn = odbc.connect('DRIVER={SQL Server}; SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password, 
                    trusted_connection='no'
                    )
cursor = cnxn.cursor()

query = '''select 
        convert(INT, bomkey) as bomkey,
        CONVERT(INT, sequenceno) as SeQuenceno,
         CONVERT(INT, ActivityFunctionKey) as ActivityFunctionKey,
         CONVERT(INT, DurationMinutes) as DurationSeconds
         
FROM [conestoga].[dbo].[DimBOMActivitiesFunctions]
 where BomKey>10001
 '''
 
seq = pd.read_sql(query, cnxn)
seq

# Maximum Capacity
# cap=pd.read_excel("ActiviActivity Cap Modties.xlsx", sheet_name='Activity Cap')
# cap=pd.read_excel("Files/Activities_adjust.xlsx", sheet_name='Activity Cap')
cap=pd.read_excel("Q:\Operations\Industrial Engineering\Maxim\Packaging tool\Activities_adjust.xlsx", sheet_name='Activity Cap Mod')

# Products and Meat Relationship Table
# ProdMeat=pd.read_excel("Files/Activities.xlsx", 
#                     #    sheet_name='Products and Meat'
#                     sheet_name='dataProdMeat'
#                        )
# print(ProdMeat.shape)

server = 'cmpcsb01'
database = 'conestoga'
username = 'remotequery'
password = "excel"
cnxn = odbc.connect('DRIVER={SQL Server}; SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password, 
                    # trusted_connection='yes'
                    )
cursor = cnxn.cursor()
# print(cursor)
query = '''SELECT 
           CAST(Itemkey as int) as FinishedGoodProductCode, 
           --Pieces, 
           Cast(ItemSort4Key as INT) as MeatItem
            FROM dimitem
            WHERE Itemkey >  ? and Itemkey < ?
            --and ItemSort4Key > 0
            '''
ProdMeat = pd.read_sql(query, cnxn, 
                   params=['10000', '39999']
                )

# ProdMeat.head()
# print("ss", ProdMeat.shape)


# Four Priority Requirement
# req = pd.read_excel("Files/req wihtout byproducts.xlsx")
req = pd.read_excel("Q:\Operations\Industrial Engineering\Maxim\Packaging tool\Files/req wihtout byproducts.xlsx", sheet_name='data (2)')
# req = pd.read_excel("Files/Cut Sheet 20221024", sheet_name='Sheet2' )

# Priority Meat Input
# meat=pd.read_excel("Files/Meats.xlsx",  sheet_name ='Priority Meat Input') #delected the byproduct,
# meat=pd.read_excel("Files/Meats_adjust_v3.xlsx",  sheet_name ='Priority Meat Input') #delected the byproduct,
# meat=pd.read_excel("Files/Meats_adjust_v3.xlsx",  sheet_name ='meat prior') #delected the byproduct,

import pyodbc as odbc
import pandas as pd
server = 'cmpcsb01'
database = 'packagingplanner'
username = 'remotequery'
password = "excel"
cnxn = odbc.connect('DRIVER={SQL Server}; SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password, 
                    trusted_connection='no'
                    )
cursor = cnxn.cursor()
print(cursor)

query = '''SELECT 

cast(MeatItem as INT) as Meat, 
SUM(P1) AS 'Priority 1', 
SUM(P2) AS 'Priority 2', 
SUM(P3) AS 'Priority 3', 
SUM(P4) AS 'Priority 4'
FROM     vwPrepForCutInputPriorities
where productiondate= ? and MeatItem < ?
GROUP BY MeatItem'''
meat = pd.read_sql(query, cnxn, 
                   params=['20221021', 800]
                )


# Cool


# Hourly Meat Input
# meat_hourly = pd.read_excel("Files/hourly_meat_input.xlsx",index_col = 0)

meat_hourly = pd.read_excel(
# "Files/hourly_meat_input_adjustment_v3.xlsx",index_col = 0, sheet_name='Sheet1'
"Q:\Operations\Industrial Engineering\Maxim\Packaging tool\Files/PackagingSimulationInputs v4.xlsx", index_col = 0, sheet_name='Summ'
 )


# relationship between hour and priority
## key represents hour, value represents priority
hour_priority={
    6:1,
    7:1,
    8:1,
    9:1,
    10:1,
    11:1,
    12:1,
    13:1,
    14:1,
    15:2,
    16:2,
    17:3,
    18:3,
    19:4,
    20:4,
    21:4,
    22:4
    }


C:\Users\msmetanin\AppData\Local\Temp\ipykernel_11988\3047029218.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  seq = pd.read_sql(query, cnxn)
C:\Users\msmetanin\AppData\Local\Temp\ipykernel_11988\3047029218.py:54: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ProdMeat = pd.read_sql(query, cnxn,
C:\Users\msmetanin\AppData\Local\Temp\ipykernel_11988\3047029218.py:94: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  meat = pd.read_sql(query, cnxn,


In [87]:
meat.Meat.nunique()

45

In [89]:
server = 'cmpcsb01'
database = 'packagingplanner'
username = 'remotequery'
password = "excel"
cnxn = odbc.connect('DRIVER={SQL Server}; SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password,
                    trusted_connection='no'
                    )
cursor = cnxn.cursor()
print(cursor)

query = '''SELECT

MeatItem,
--ItemKey,
ProductionDate,
P1,
P2,
P3,
P4,
COALESCE(P1,0)
         + COALESCE(P2,0)
         + COALESCE(P3,0)
         + COALESCE(P4,0)
 as Total
FROM     vwPrepForCutInputPriorities
where productiondate= ? and MeatItem < ?
--GROUP BY MeatItem
'''
meat1 = pd.read_sql(query, cnxn,
                   params=['20221021', 899]
                )
print(meat1.MeatItem.nunique())
meat1.sort_values(by = 'MeatItem')

46


C:\Users\msmetanin\AppData\Local\Temp\ipykernel_11988\2001958394.py:29: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  meat1 = pd.read_sql(query, cnxn,


,MeatItem,ProductionDate,P1,P2,P3,P4,Total
0,101.0,20221021,0,0,6,0,6
1,103.0,20221021,0,930,0,1860,2790
2,103.0,20221021,0,8680,0,0,8680
3,104.0,20221021,0,0,4,12,16
4,104.0,20221021,0,0,50,24,74
...,...,...,...,...,...,...,...
77,762.0,20221021,2500,0,0,0,2500
78,775.0,20221021,0,0,584,808,1392
79,786.0,20221021,0,0,0,460,460
80,787.0,20221021,0,0,0,200,200


In [88]:
m1 = set(meat['Meat'])
m2 = set(meat1.MeatItem)

diff = [x for x in m1 if x not in m2]
diff

[]

In [8]:
seq.loc[seq.ActivityFunctionKey == 6789]

,bomkey,SeQuenceno,ActivityFunctionKey,DurationSeconds
120,10033,1,6789,17
251,10133,1,6789,44
364,10333,1,6789,17
437,10443,1,6789,42
664,11025,1,6789,17
...,...,...,...,...
4219,31032,1,6789,129
4228,31048,1,6789,84
4285,32043,1,6789,84
4607,50016,1,6789,17


In [6]:
seq

,bomkey,SeQuenceno,ActivityFunctionKey,DurationSeconds
0,10002,1,6718,24
1,10002,2,6770,171
2,10002,3,6753,86
3,10002,4,4753,85
4,10002,5,6705,10
...,...,...,...,...
4782,80069,3,6716,80
4783,80069,4,6721,18
4784,80069,5,6705,10
4785,80069,6,6722,55


In [7]:
hour_priority.keys()

dict_keys([6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22])

In [8]:
seq.head(2)

,bomkey,SeQuenceno,ActivityFunctionKey,DurationSeconds
0,10002,1,6718,24
1,10002,2,6770,171


In [9]:
cap.head(2
         )

,Activity_Num,Activity_Desc,Max_Dom,Chilled pork,Max
0,6701,Pkg - Box Backribs CC,6,0,6
1,6703,Pkg - C105 Ham - Open Bags,0,0,1


In [10]:
ProdMeat.head(2)

,FinishedGoodProductCode,MeatItem
0,10001,324
1,10002,307


In [11]:
new_var = req.head(2)
new_var

,itemkey,itemdesc,PiecesInBox,Priority1Qty,Priority2Qty,Priority3Qty,Priority4Qty
0,34019,Fresh Pork Ham Boneless Inside Denuded,450,0,0,4,2
1,34119,Fresh Pork Hams Bnls Inside Denuded,450,0,6,0,0


In [12]:
meat.head(2)

,Meat,Priority 1,Priority 2,Priority 3,Priority 4
0,101,0,0,6,0
1,103,0,9610,0,1860


In [13]:
meat_hourly.fillna(0, inplace = True)
meat_hourly.head(2)

,701,705,706,767,702,710,712,714,719,760,...,341,313,320,352,348,309,401,405,403,402
t,,,,,,,,,,,,,,,,,,,,,
6,0,400,0,0,300,0,0,400,0,28,...,0,0,0,0,770,0,770,770,770,0
7,0,400,0,0,300,0,0,400,0,0,...,0,0,0,0,770,0,770,770,770,0


In [4]:
req.head()

,itemkey,itemdesc,PiecesInBox,Priority1Qty,Priority2Qty,Priority3Qty,Priority4Qty
0,34019,Fresh Pork Ham Boneless Inside Denuded,450,0,0,4,2
1,34119,Fresh Pork Hams Bnls Inside Denuded,450,0,6,0,0
2,30016,Fresh Pork Side Rib,415,0,15,1,19
3,30008,Fresh Pork Shoulder Picnic Boneless,270,0,0,0,4
4,31508,Fresh Pork Shoulder Picnic Boneless,270,0,9,0,12


In [15]:

req = req.loc[req["PiecesInBox"]!=0,:]
req.head(3)

,itemkey,itemdesc,PiecesInBox,Priority1Qty,Priority2Qty,Priority3Qty,Priority4Qty
0,34019,Fresh Pork Ham Boneless Inside Denuded,450,0,0,4,2
1,34119,Fresh Pork Hams Bnls Inside Denuded,450,0,6,0,0
2,30016,Fresh Pork Side Rib,415,0,15,1,19


In [16]:
meat.Meat.values

array([101, 103, 104, 111, 141, 144, 147, 148, 149, 303, 309, 313, 324,
       341, 348, 401, 402, 403, 405, 501, 502, 506, 510, 511, 518, 545,
       553, 583, 701, 702, 703, 705, 706, 710, 712, 714, 715, 716, 719,
       739, 760, 762, 775, 786, 787], dtype=int64)

In [17]:
meat_hourly.shape

(17, 53)

In [18]:
import numpy as np
# meat_hourly.count(axis= 1)
meat_hourly.columns = np.dtype('int64').type(meat_hourly.columns)
meat_hourly.head(2)

,701,705,706,767,702,710,712,714,719,760,...,341,313,320,352,348,309,401,405,403,402
t,,,,,,,,,,,,,,,,,,,,,
6,0,400,0,0,300,0,0,400,0,28,...,0,0,0,0,770,0,770,770,770,0
7,0,400,0,0,300,0,0,400,0,0,...,0,0,0,0,770,0,770,770,770,0


In [19]:
# meat_hourly = meat_hourly.loc[:,list(meat.Meat.values)]
# meat_hourly.head(2)

In [20]:
meat_hourly

,701,705,706,767,702,710,712,714,719,760,...,341,313,320,352,348,309,401,405,403,402
t,,,,,,,,,,,,,,,,,,,,,
6,0,400,0,0,300,0,0,400,0,28,...,0,0,0,0,770,0,770,770,770,0
7,0,400,0,0,300,0,0,400,0,0,...,0,0,0,0,770,0,770,770,770,0
8,0,400,0,0,300,0,0,400,0,0,...,0,370,0,0,400,0,770,770,260,510
9,0,400,0,0,300,0,0,400,0,0,...,0,770,0,0,0,0,770,770,0,770
10,4,400,0,0,260,0,140,400,0,0,...,0,770,0,0,0,0,770,770,0,770
11,4,400,0,0,0,0,400,400,0,0,...,0,732,0,0,0,38,770,770,0,770
12,4,400,0,0,0,0,800,0,0,0,...,0,0,0,0,0,770,770,770,0,770
13,0,281,300,0,0,0,800,0,0,0,...,0,0,0,0,0,770,770,770,0,770
14,0,0,150,0,0,0,200,0,0,0,...,0,0,0,0,0,770,770,770,0,770


In [21]:
meat.Meat.values

array([101, 103, 104, 111, 141, 144, 147, 148, 149, 303, 309, 313, 324,
       341, 348, 401, 402, 403, 405, 501, 502, 506, 510, 511, 518, 545,
       553, 583, 701, 702, 703, 705, 706, 710, 712, 714, 715, 716, 719,
       739, 760, 762, 775, 786, 787], dtype=int64)

# Data Cleaning


In [22]:
# Check capitalization of the columns naem
seq.columns=['BomKey' if x=='bomkey' else x for x in seq.columns]

#Some products have 0 PiecesInBox, these skus are also excluded
req = req.loc[req["PiecesInBox"]!=0,:]

# exclude byproduct(just include meat_id in Meat table)
meat_hourly = meat_hourly.loc[:,list(meat.Meat.values)]

#combine meat inputs and requirements
meat_and_req = req.merge(ProdMeat, left_on='itemkey',  right_on='FinishedGoodProductCode', how='left')

In [23]:
# sales.drop(sales.CustomerID.isin(badcu))
chilled_pork = [148, 152, 348, 352, 353, 460, 461, 702, 762, 784, 786, 787]
meat_and_req[~meat_and_req['MeatItem'].isin(chilled_pork)].head()

,itemkey,itemdesc,PiecesInBox,Priority1Qty,Priority2Qty,Priority3Qty,Priority4Qty,FinishedGoodProductCode,MeatItem
0,34019,Fresh Pork Ham Boneless Inside Denuded,450,0,0,4,2,34019,553
1,34119,Fresh Pork Hams Bnls Inside Denuded,450,0,6,0,0,34119,553
2,30016,Fresh Pork Side Rib,415,0,15,1,19,30016,111
3,30008,Fresh Pork Shoulder Picnic Boneless,270,0,0,0,4,30008,715
4,31508,Fresh Pork Shoulder Picnic Boneless,270,0,9,0,12,31508,715


In [24]:
meat_hourly[meat_hourly.columns[~meat_hourly.columns.isin(chilled_pork)]]

,101,103,104,111,141,144,147,149,303,309,...,706,710,712,714,715,716,719,739,760,775
t,,,,,,,,,,,,,,,,,,,,,
6,6,900,0,940,0,0,0,150,330,0,...,0,0,0,400,165,0,0,0,28,0
7,0,900,0,940,0,0,0,150,330,0,...,0,0,0,400,265,0,0,0,0,0
8,0,900,0,940,0,0,0,150,330,0,...,0,0,0,400,473,0,0,0,0,0
9,0,860,0,940,0,0,0,190,330,0,...,0,0,0,400,700,0,0,0,0,0
10,0,850,0,940,0,0,0,300,330,0,...,0,0,140,400,700,0,0,0,0,0
11,0,850,0,940,0,0,0,300,330,38,...,0,0,400,400,700,0,0,0,0,0
12,0,850,0,940,0,0,0,600,330,770,...,0,0,800,0,1100,0,0,0,0,0
13,0,850,0,940,0,0,0,600,330,770,...,300,0,800,0,1100,0,0,0,0,0
14,0,900,0,487,0,0,0,0,330,770,...,150,0,200,0,457,60,0,0,0,0


In [25]:
meat_hourly

,101,103,104,111,141,144,147,148,149,303,...,714,715,716,719,739,760,762,775,786,787
t,,,,,,,,,,,,,,,,,,,,,
6,6,900,0,940,0,0,0,250,150,330,...,400,165,0,0,0,28,500,0,0,0
7,0,900,0,940,0,0,0,250,150,330,...,400,265,0,0,0,0,400,0,0,0
8,0,900,0,940,0,0,0,250,150,330,...,400,473,0,0,0,0,400,0,0,0
9,0,860,0,940,0,0,0,251,190,330,...,400,700,0,0,0,0,400,0,0,0
10,0,850,0,940,0,0,0,251,300,330,...,400,700,0,0,0,0,400,0,0,0
11,0,850,0,940,0,0,0,251,300,330,...,400,700,0,0,0,0,400,0,0,0
12,0,850,0,940,0,0,0,251,600,330,...,0,1100,0,0,0,0,0,0,0,0
13,0,850,0,940,0,0,0,246,600,330,...,0,1100,0,0,0,0,0,0,0,0
14,0,900,0,487,0,0,0,0,0,330,...,0,457,60,0,0,0,0,0,0,0


In [26]:
# Write loop to exlude for missing meat items

meat_hourly.loc[10:20, [148, 702, 762, 786, 787]]

,148,702,762,786,787
t,,,,,
10,251,260,400,0,0
11,251,0,400,0,0
12,251,0,0,0,0
13,246,0,0,0,0
14,0,0,0,0,0
15,0,0,0,0,0
16,0,0,0,0,0
17,0,0,0,0,0
18,0,0,0,0,0


In [27]:
max(meat_hourly.index)

22

In [28]:
meat_hourly = meat_hourly.loc[:,list(meat.Meat.values)]
meat_hourly

,101,103,104,111,141,144,147,148,149,303,...,714,715,716,719,739,760,762,775,786,787
t,,,,,,,,,,,,,,,,,,,,,
6,6,900,0,940,0,0,0,250,150,330,...,400,165,0,0,0,28,500,0,0,0
7,0,900,0,940,0,0,0,250,150,330,...,400,265,0,0,0,0,400,0,0,0
8,0,900,0,940,0,0,0,250,150,330,...,400,473,0,0,0,0,400,0,0,0
9,0,860,0,940,0,0,0,251,190,330,...,400,700,0,0,0,0,400,0,0,0
10,0,850,0,940,0,0,0,251,300,330,...,400,700,0,0,0,0,400,0,0,0
11,0,850,0,940,0,0,0,251,300,330,...,400,700,0,0,0,0,400,0,0,0
12,0,850,0,940,0,0,0,251,600,330,...,0,1100,0,0,0,0,0,0,0,0
13,0,850,0,940,0,0,0,246,600,330,...,0,1100,0,0,0,0,0,0,0,0
14,0,900,0,487,0,0,0,0,0,330,...,0,457,60,0,0,0,0,0,0,0


In [29]:
meat_hourly

,101,103,104,111,141,144,147,148,149,303,...,714,715,716,719,739,760,762,775,786,787
t,,,,,,,,,,,,,,,,,,,,,
6,6,900,0,940,0,0,0,250,150,330,...,400,165,0,0,0,28,500,0,0,0
7,0,900,0,940,0,0,0,250,150,330,...,400,265,0,0,0,0,400,0,0,0
8,0,900,0,940,0,0,0,250,150,330,...,400,473,0,0,0,0,400,0,0,0
9,0,860,0,940,0,0,0,251,190,330,...,400,700,0,0,0,0,400,0,0,0
10,0,850,0,940,0,0,0,251,300,330,...,400,700,0,0,0,0,400,0,0,0
11,0,850,0,940,0,0,0,251,300,330,...,400,700,0,0,0,0,400,0,0,0
12,0,850,0,940,0,0,0,251,600,330,...,0,1100,0,0,0,0,0,0,0,0
13,0,850,0,940,0,0,0,246,600,330,...,0,1100,0,0,0,0,0,0,0,0
14,0,900,0,487,0,0,0,0,0,330,...,0,457,60,0,0,0,0,0,0,0


In [30]:
list(meat.Meat.values)
meat_hourly.loc[:, 101]

t
6     6
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    0
18    0
19    0
20    0
21    0
22    0
Name: 101, dtype: int64

In [31]:
meat_and_req.columns

Index(['itemkey', 'itemdesc', 'PiecesInBox', 'Priority1Qty', 'Priority2Qty',
       'Priority3Qty', 'Priority4Qty', 'FinishedGoodProductCode', 'MeatItem'],
      dtype='object')

In [32]:
meat_hourly.head(2)

,101,103,104,111,141,144,147,148,149,303,...,714,715,716,719,739,760,762,775,786,787
t,,,,,,,,,,,,,,,,,,,,,
6,6,900,0,940,0,0,0,250,150,330,...,400,165,0,0,0,28,500,0,0,0
7,0,900,0,940,0,0,0,250,150,330,...,400,265,0,0,0,0,400,0,0,0


# Feasibility Check For 4 Priority Plan


### - Meat Feasibility Check


In [33]:
meat_hourly.isna().value_counts().sum()

17

In [34]:
ProdMeat.tail(2)
# req.head(2)
# meat.head(2)

,FinishedGoodProductCode,MeatItem
1223,39200,515
1224,39211,563


In [35]:
# new_req = req.merge(ProdMeat, left_on='itemkey',  
#                     right_on='FinishedGoodProductCode', 
#                     how='left', 
#                     # validate="1:m"
#                                    )
# new_req[['FinishedGoodProductCode', 'MeatItem']] = new_req[['FinishedGoodProductCode', 'MeatItem']]
# # .fillna(0).astype('int64')
# new_req.loc[new_req['MeatItem'] <= 899]

# Function Meat Piece Check


## Onme MEat Check Function


In [36]:
meat.head(2)    

,Meat,Priority 1,Priority 2,Priority 3,Priority 4
0,101,0,0,6,0
1,103,0,9610,0,1860


### One meat checking function

In [37]:
# result = []
# meat_id_list = list(req.merge(ProdMeat, 
#                                   left_on='itemkey',  
#                                   right_on='FinishedGoodProductCode', 
#                                   how='left',
#                                validate = '1:m' # validates one to many connection
#                                ).MeatItem.drop_duplicates())
# for meat_id in meat_id_list:
#     new_req = req.merge(ProdMeat, 
#                             left_on='itemkey',  
#                             right_on='FinishedGoodProductCode', 
#                             how='left',
#                             validate= "1:m"  # validates one to many connection
#                             )

#     new_req = new_req.loc[new_req['MeatItem']==meat_id,:]
#     pieces_need_df = new_req.copy()
#     priority_cols = ['Priority1Qty', 'Priority2Qty', 'Priority3Qty', 'Priority4Qty']
#     pieces_priority_cols = []
#     for p in range(1,4+1):
#         pieces_need_df['pieces_p'+str(p)] = pieces_need_df['PiecesInBox']*pieces_need_df[priority_cols[p-1]]
#         pieces_priority_cols.append('pieces_p'+str(p))
#     pieces_need_df.drop(priority_cols, axis=1, inplace=True)
#     pieces_need_df.drop(['FinishedGoodProductCode'], axis=1, inplace=True)
#     #requirements
#     p1_req = pieces_need_df.pieces_p1.sum()
#     p2_req = pieces_need_df.pieces_p2.sum()
#     p3_req = pieces_need_df.pieces_p3.sum()
#     p4_req = pieces_need_df.pieces_p4.sum()
#     # print(p2_req)
#     avaliable=meat.loc[meat['Meat']==meat_id, :]
#     # print(avaliable)
#     p1_ava = avaliable.loc[:,'Priority 1'].values[0]
#     # print(p1_ava)
#     p2_ava = avaliable.loc[:,'Priority 2'].values[0]
#     p3_ava = avaliable.loc[:,'Priority 3'].values[0]
#     p4_ava = avaliable.loc[:,'Priority 4'].values[0]
#     p1_remaining=p1_ava-p1_req   
#     p2_remaining=(p1_remaining+p2_ava)-p2_req
#     p3_remaining=(p2_remaining+p3_ava)-p3_req
#     p4_remaining=(p3_remaining+p4_ava)-p4_req
    
#     if p1_remaining<0:
#         dic={'meat': meat_id,'priority':'p1n'}
#         dic['needed_pieces'] = p1_remaining*(-1)
#         result.append(dic)
#         # print(len(result))
#     if p2_remaining<0:
#         dic={'meat': meat_id,'priority':'p2'}
#         dic['needed_pieces'] = p2_remaining*(-1)
#         result.append(dic)
#         # print(len(result))
#     if p3_remaining<0:
#         dic={'meat': meat_id,'priority':'p3'}
#         dic['needed_pieces'] = p3_remaining*(-1)
#         result.append(dic)
#         # print(len(result))
#     if p4_remaining<0:
#         dic={'meat': meat_id,'priority':'p4'}
#         dic['needed_pieces'] = p4_remaining*(-1)
#         result.append(dic)
        # print(len(result))    
# result

### check Function #2 actvity time check

In [38]:
# req_transpose=req.loc[:,["itemkey",
#                              "Priority1Qty",
#                              "Priority2Qty",
#                              "Priority3Qty",
#                              "Priority4Qty"]]
# req_transpose=req_transpose.set_index('itemkey')
# req_transpose = req_transpose.stack(0).rename_axis(
#         ('itemkey','priority')).reset_index()
# req_transpose
# new_seq = seq.merge(req_transpose, left_on='BomKey',  right_on='itemkey', how='right')
# new_seq


In [39]:


# meat_id_list = list(req.merge(ProdMeat, 
#                               left_on='itemkey',  
#                                 right_on='FinishedGoodProductCode', 
#                                 how='left').MeatItem.drop_duplicates()#.fillna(0).astype('int64')
# )
# meat_id_list['FinishedGoodProductCode']
# .MeatItem.drop_duplicates()
# meat_id_list.astype("int64")
# meat_id_list = meat_id_list.fillna(0, inplace = True)
# meat_id_list = meat_id_list.fillna(0).astype('int64')
# 
# meat_id_list

In [40]:
# new_req = req.merge(ProdMeat, left_on='itemkey',  right_on='FinishedGoodProductCode', how='left')
# new_req = new_req['FinishedGoodProductCode', 'MeatItem'].astype('int64')

In [41]:
# www = req.merge(ProdMeat, 
#                                   left_on='itemkey',  
#                                   right_on='FinishedGoodProductCode', 
#                                   how='left',
#                                validate = '1:m'
#                                )
# www
# www.loc[www['MeatItem']==meat_id,:]


In [42]:
# from GenerateHourlyPlan import *
# meat_pieces_check(ProdMeat,req, meat)

In [43]:

# meat_id_list = list(req.merge(ProdMeat, 
#                               left_on='itemkey',  
#                               right_on='FinishedGoodProductCode', 
#                               how='left',
#                              validate="1:m"
#                                ).MeatItem.drop_duplicates()#.fillna(0).astype('int64')
#                     )
# for meat_id in meat_id_list:
#     #if meat_id not in [412,803,901,902,413]:
#     result = one_meat_checking(meat_id=meat_id,
#                                result=result,
#                                ProdMeat=ProdMeat, 
#                                meat=meat,
#                                req=req
#                                )
# print(meat_id)

In [44]:
# result = []
# meat_id_list = list(req.merge(ProdMeat, 
#                               left_on='itemkey',  
#                               right_on='FinishedGoodProductCode', 
#                               how='left',
#                               validate='1:m',
#                               ).MeatItem.drop_duplicates()#.fillna(0).astype('int64')
#                     )
# for meat_id in meat_id_list:
#     #if meat_id not in [412,803,901,902,413]:
#     result = one_meat_checking(meat_id=meat_id,
#                                result=result,
#                                ProdMeat=ProdMeat, 
#                                meat=meat,
#                                req=req)
# print(len(result))

In [45]:
# avaliable=meat.loc[meat['Meat']==meat_id, :]
# avaliable.loc[:,'Priority 1'].values[0]


In [46]:

# avaliable=meat.loc[meat['Meat']==meat_id, :]

# avaliable

In [47]:
# meat_pieces_check(ProdMeat,req, meat)


### - Activity Time Checking-accumulated


In [48]:
activity_time_check(req,hour_priority,cap,seq)

Infeasible


,Activity,priority,needed_time
0,6711.0,p2,30702.0
1,6711.0,p3,38313.0
2,6711.0,p4,54567.0
3,6741.0,p3,38.0
4,6741.0,p4,1748.0
5,6746.0,p3,2700.0
6,6746.0,p4,2700.0


In [49]:
acticity_skus = {}
for activity_id in cap['Activity_Num'].values:
    acticity_skus[activity_id] = list(set(seq.loc[seq['ActivityFunctionKey']==activity_id,'BomKey']))
acticity_skus.keys()


dict_keys([6701, 6703, 6705, 6714, 6715, 6716, 6717, 6718, 6720, 6721, 6722, 6724, 6726, 6729, 6741, 6746, 6753, 6756, 6758, 6759, 6760, 6762, 6763, 6770, 6711, 6719, 6730, 6740, 6755])

# Plan Break Down and Get the best Hourly Plan


### After meat feasibility check and activity time check-accumulated for 4 priority plan, plan break down function can be conducted.


In [50]:
# meat_and_req
# meat_hourly
# hour_priority
# cap
# seq
# req.loc[req['itemkey'] < 30000]

# method1_adjusting_rate_list
# method2_combo_ratio_list
# method2_adjusting_rate_lis

In [51]:
len([])

0

In [52]:
meat_pieces_check(ProdMeat, req, meat)

    Meat  Priority 1  Priority 2  Priority 3  Priority 4
26   553           0        2700        1800         900
   Meat  Priority 1  Priority 2  Priority 3  Priority 4
3   111           0        6225        2350        8032
    Meat  Priority 1  Priority 2  Priority 3  Priority 4
36   715           0        5130         290        8520
    Meat  Priority 1  Priority 2  Priority 3  Priority 4
39   739           0           0         240           0
    Meat  Priority 1  Priority 2  Priority 3  Priority 4
32   706           0           0        1040         756
    Meat  Priority 1  Priority 2  Priority 3  Priority 4
11   313           0         225         282        2135
    Meat  Priority 1  Priority 2  Priority 3  Priority 4
40   760          28           0           0         411
    Meat  Priority 1  Priority 2  Priority 3  Priority 4
30   703           0           0         241         856
   Meat  Priority 1  Priority 2  Priority 3  Priority 4
5   144           0           0   

'Feasible'

In [53]:
# plan_break_down(meat_and_req, meat_hourly, hour_priority)

In [54]:
%%time
method1_adjusting_rate_list=[0.01,0.03]
method2_combo_ratio_list=[0.3,0.7]
method2_adjusting_rate_list=[0.01]
result, best_plan=get_best_plan(meat_and_req,
                                meat_hourly,
                                hour_priority,
                                cap,
                                seq,
                                req,
                                method1_adjusting_rate_list,
                                method2_combo_ratio_list,
                                method2_adjusting_rate_list)

method1 - 0.01
time_comparison      hour  Activity_Num  time_required  hourly_available_seconds  \
3       6          6715        62442.0                     46800   
8       6          6721        44808.0                     25200   
17      7          6705         7340.0                      7200   
19      7          6715        58220.0                     46800   
24      7          6721        43485.0                     25200   
35      8          6715        50401.0                     46800   
40      8          6721        35186.0                     25200   
187    17          6726        19648.0                     18000   

     time_still_need  
3            15642.0  
8            19608.0  
17             140.0  
19           11420.0  
24           18285.0  
35            3601.0  
40            9986.0  
187           1648.0  
     hour  Activity_Num  time_required  hourly_available_seconds  \
3       6          6715        62442.0                     46800   
8       6    

KeyError: 'p2_cum'

In [55]:
# infeasible_table2[:1].Activity_Num.values[0]

In [56]:
hourly_activity_time_check(new_plan,cap,hour_priority,seq)

NameError: name 'new_plan' is not defined

## meat_split_combo_pkg

In [ ]:
hourly_req_melt=pd.melt(hourly_req,id_vars='itemkey',value_vars=hour_priority.keys())

hourly_req_melt.columns = ['itemkey','hour','sku_num']

type(hourly_req_melt.hour.values)

# infeasible_table[:1].hour.values[0]

## Function #5

In [ ]:
# hourly_req, _ = plan_break_down(meat_and_req=meat_and_req, 
#                                             meat_hourly=meat_hourly, 
#                                             hour_priority=hour_priority)


## Hourly Activity Time Check

# Show the Hourly Total Position Num for the Best Plan


In [ ]:
hourly_total_position_stdev(best_plan,cap,hour_priority,seq,plot=True)

# Show OEE Table for the Best Plan


In [ ]:
OEE, OEE_table=EEO_caculation(best_plan,cap,hour_priority,seq)
OEE_table

# Estimated Position Num for Each Activity (Best Plan)


In [ ]:
visualize_estimated_position_num(best_plan,cap,hour_priority,seq)

In [ ]:
best_plan.to_excel("best_plan.xlsx")

# More Detailed Analysis


### To see how adjustment helps improve the overall efficiency.


##### Use Method 1 get an hourly plan

Method 1 calculate ratios based on pieces required by each sku.


In [ ]:
hourly_packaging_plan_method1, requirements_not_complete_method1 = plan_break_down(meat_and_req, meat_hourly, hour_priority)
hourly_packaging_plan_method1

#### Adjust hourly plan with given learning rate


In [ ]:
new_plan,new_infeasible_table = adjust_infeasible_plan(hourly_packaging_plan_method1,req,cap,seq,hour_priority,adjusting_rate=0.01)

#### Hourly Total Position Num Before Adjustment


In [ ]:
hourly_total_position_stdev(hourly_packaging_plan_method1,cap,hour_priority,seq,plot=True)

#### Hourly Total Position Num after Adjustment


In [ ]:
hourly_total_position_stdev(new_plan,cap,hour_priority,seq,plot=True)